## Простая сверточная сеть. 

* Создадим функции, генерирующие веса сети.
* Содадим слои сети.
* Запустим обучение.
* Посмотрим, как влияет инициализация параметров на обучение.


In [ ]:
import tensorflow as tf 
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data

import sys

Для удобства экспериментов создадим функции, которые будут генерировать веса. Порог сделаем небольшим положительным числом. Остальные веса сгенерируем "урезанным" нормальным распределением.

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    
    # Хорошо подумайте, прежде, чем делать так (!):
    # initial = tf.random_normal(shape, 0, 4.0)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    
    # Не надо делать так (!):
    # initial = tf.random_normal((), 0, 4.0)
    return tf.Variable(initial)


Определим слои сети.

In [ ]:
def conv2d(x, W):
    # strides: по умолчанию порядок [Примеры, Вертикальный шаг, Горизонтальный шак, каналы]
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W) + b)


In [ ]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b


Строим вычислительный граф:

In [ ]:
# Входное изображение --  1х784 вектор 
x = tf.placeholder(tf.float32, shape=[None, 784])

# Ответы
y_ = tf.placeholder(tf.float32, shape=[None, 10])


# Нужно переформатировать входы под ожидаемый формат входных данных
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Первый сверточный слой
conv1 = conv_layer(x_image, shape=[5, 5, 1, 32])
conv1_pool = max_pool_2x2(conv1)

# Второй сверточный слой
conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

# Чтобы закинуть данные в полносвязную сеть, нужно сделать их "плоскими"
conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

# Прореживание (Dropout) нужно для больших сетей
keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

# Выходной слой 
y_conv = full_layer(full1_drop, 10)


Создадим сеть с Softmax выходом, в качестве функции потерь возмем перекрестную энтропию. Вычисляем точность.

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Запускаем модель на выполнение.

In [ ]:
DATA_DIR = '/tmp/data' if not 'win32' in sys.platform else "c:\\tmp\\data"
STEPS = 1000
MINIBATCH_SIZE = 50
mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)

In [ ]:
sess =  tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(STEPS):
    batch = mnist.train.next_batch(MINIBATCH_SIZE)

    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    if i % 200 == 0:
        valid_accuracy = sess.run(accuracy, 
                                  feed_dict={x: mnist.validation.images, 
                                             y_: mnist.validation.labels,
                                             keep_prob: 1.0})
        print("step {}, точность на валидац. множестве {}".format(i, valid_accuracy))


In [ ]:

X = mnist.test.images 
Y = mnist.test.labels 
test_accuracy = np.mean([sess.run(accuracy, feed_dict={x:X, y_:Y, 
                                                       keep_prob:1.0})])

print("точность на тестовом множестве: {:.4f}%".format(test_accuracy*100.))

In [ ]:
sess.close()